# Исследование надёжности заёмщиков

Цель проекта - разобраться, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты проекта будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Для начала считаем данные и посмотрим на них

In [1]:
import pandas as pd

data = pd.read_csv('credits.csv')
data.head(10)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Первые выводы о данных

* children:
    - тип данных int соответствует данным в столбце
    - пропусков нет
* days_employed:
    - тип данных должен быть тоже int (сейчас float)
    - не ясно, что значат отрицательные значения
    - есть пропуски данных, которые будем заполнять средними значениями
* dob_days:
    - название не соответствует содержанию, лучше переименовать для дальнейшей работы
    - тип данных int соответствует данным в столбце
    - пропусков нет
* education:
    - тип данных соответствует данным в столбце
    - пропусков(т.е. NaN) нет, надо смотреть нет ли 'пкстых значений' (например, 'нет данных' и.т.п.)
    - надо привести все к строчному написанию (т.е. СРЕДНЕЕ -> среднее)
    - можно вынести в словарь вместе с education_id
* education_id:  
    - тип данных int соответствует данным в столбце
    - пропусков нет
    - категориальная переменная, будет ключом к словарю education_dict
* family_status:
    - тип данных соответствует данным в столбце
    - пропусков(т.е. NaN) нет, надо смотреть нет ли 'пкстых значений' (например, 'нет данных' и.т.п.)
    - надо проверять все-ли одинаковые статусы написаны одинаков, если нет, то приводить к единому написанию
    - можно вынести в словарь вместе с family_status_id
* family_status_id:
    - тип данных int соответствует данным в столбце
    - пропусков нет
    - категориальная переменная, будет ключом к словарю family_status_dict
* gender:
    - тип данных соответствует данным в столбце, хотя надо смотреть по задаче в дальнейшем. Иногда удобно переводить такие категории в бинарный вид (например, мужской - 1, женский - 0) 
    - пропусков нет
* income_type:
    - тип данных соответствует данным в столбце
    - пропусков(т.е. NaN) нет, надо смотреть нет ли 'пкстых значений' (например, 'нет данных' и.т.п.)
    - надо проверять все-ли одинаковые типы написаны одинаков, если нет, то приводить к единому написанию
    - возможно надо ввести кодировку и сделать словарь income_type_dict
* debt:
    - тип данных int соответствует данным в столбце
    - пропусков нет
    - бинарная характеристика
* total_income:
    - тип данных соответствует данным в столбце
    - надо проверить на неясные значения (например, отрицательные или оооочень большие)
    - есть пропуски данных, которые будем заполнять средними значениями (возможно нужно заполнять среднип по группам, например, income_type)
* purpose - надо смотреть стандартные там формулироваки или вольный текст:
    - если формалировки стандарные, то их нужно дополнительно унифицировать и, возможно, вынести в словарь
    - если там свободный текст, то нужно проводили лемматизацию (выделять главное из набора слов)

## Шаг 1. 

Удалим столбец days_employed, так как данные в нем не соответстуют назначению.
В дальнейшем можно уточнить у "владельца" данных, что кладется в этот столбец.

Также переименуем столбец dob_days в age, что лучше отражает суть данных в нем 

In [3]:
data = data.drop(['days_employed'], axis=1)
data = data.rename(columns={'dob_days': 'age'})
data.head()

,children,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


## Шаг 2.

Заполним пропуски в столбце total_income. Столбец вещественный. поэтому вставим среднее значение. 

In [4]:
data['total_income'].isnull().sum()

2174

In [5]:
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())
data['total_income'].isnull().sum()

0

## Шаг 3.

Выделим словари:
* education_dict
* family_status
* введем income_type_dict

Удалим дубликаты из словарей, унифицировав имена одинаковых категорий.

In [6]:
# выделим education dict
education_dict = pd.DataFrame()
education_dict['education_id']= data['education_id']
education_dict['education_name']= data['education'].str.lower()

education_dict['education_name'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_name, dtype: int64

In [7]:
# все названия в порядке, удалим дубликаты
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

#словарь готов
education_dict.head()

,education_id,education_name
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [8]:
# аналогично со словарем family_status_dict
family_status_dict = pd.DataFrame()
family_status_dict['family_status_id']= data['family_status_id']
family_status_dict['family_status_name']= data['family_status'].str.lower()

family_status_dict['family_status_name'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status_name, dtype: int64

In [9]:
# все названия в порядке удалим дубликаты
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

#словарь готов
family_status_dict.head()

,family_status_id,family_status_name
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [10]:
# сделаем заготовку для словаря income_type
income_type_dict = pd.DataFrame()
income_type_dict['income_type_name']= data['income_type'].str.lower()

income_type_dict['income_type_name'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type_name, dtype: int64

In [11]:
# удалим дубликаты и добавим индексы

income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)
income_type_dict['income_type_id'] = 0
for i in range(8):
    income_type_dict['income_type_id'][i] = i
 # словарь готов
income_type_dict.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,income_type_name,income_type_id
0,сотрудник,0
1,пенсионер,1
2,компаньон,2
3,госслужащий,3
4,безработный,4
5,предприниматель,5
6,студент,6
7,в декрете,7


In [12]:
# удалим ненужные столбцы из данных

data = data.drop(['education'], axis=1)
data = data.drop(['family_status'], axis=1)

data = data.merge(income_type_dict, left_on='income_type', right_on='income_type_name')
data = data.drop(['income_type'], axis=1)
data = data.drop(['income_type_name'], axis=1)

data.head(10)

,children,age,education_id,family_status_id,gender,debt,total_income,purpose,income_type_id
0,1,42,0,0,F,0,253875.639453,покупка жилья,0
1,1,36,1,0,F,0,112080.014102,приобретение автомобиля,0
2,0,33,1,0,M,0,145885.952297,покупка жилья,0
3,3,32,1,0,M,0,267628.550329,дополнительное образование,0
4,0,50,1,0,M,0,135823.934197,образование,0
5,2,35,0,1,F,0,95856.832424,на проведение свадьбы,0
6,0,41,1,0,M,0,144425.938277,покупка жилья для семьи,0
7,0,40,1,0,F,0,77069.234271,покупка коммерческой недвижимости,0
8,0,54,2,0,F,0,130458.228857,приобретение автомобиля,0
9,1,26,1,0,F,0,116820.904450,строительство собственной недвижимости,0


## Шаг 4.

Выделим леммы в значениях столбца с целями получения кредита.

In [13]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()
lemmas = []


for i in range(data.shape[0]):
    text = data['purpose'][i]
    for lemma in m.lemmatize(text):
        lemmas.append(lemma)
        
#print(lemmas)
print(Counter(lemmas))

ModuleNotFoundError: No module named 'pymystem3'

In [14]:
# Посмотрим нужно ли вообще леммитизировать
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Да, лемметизирование имеет смысл.

Хотя можно проанализировать и объеденить несколько обозначений в одно, например "свадьба", "сыграть свадьбу" и "на проведение свадьбы" явно одно и тоже. В то же время, с покупкой недвижимости и жилья все не так однозначно. Есть разделения на коммерческую и некоммерческую недвижимость, есть разделения жилья для сдачи и для семьи. Одновременно есть просто покупка жилья и покупка недвижимости.

Необходимо уточнять у Заказчика можно ли условно разделить на свадьба, недвижимость, автомобиль ...

В принципе можно из текущих значений сделать словарь

## Шаг 5.

Сделаем набор сводных таблиц.

In [15]:
# удалим строчку со значением children = -1
# и строчку со знаяением gender = 'XNA'

data_final = data.loc[data['children'] != -1]
data_final = data_final.loc[data['gender'] != 'XNA']
data_pivot = data_final.pivot_table(index='gender', columns='children', values='debt', aggfunc='sum')
print(data_pivot.head(10))

children   0    1    2   3   4   5   20
gender                                 
F         592  245  134  17   1   0   4
M         471  199   60  10   3   0   4


Видно, что чем больше детей, тем меньше невыплаченных кредитов.
Также, женщины чаще невыплачивают кредит, чем мужчины.

Нужно проверить, является ли эта разница статистически значимой (или людей с тремя детьми просто меньше)

Посмотрев несколько раз *value_counts()* мы знаем, что в выборке

14236 - женщин и 7288 - мужчин,
детей 0 - 14149, 1 - 4818, 2 - 2055, 3 - 330, 4 - 41, 5 - 9
соответственно получаем 

женщин, не вернувших кредит 4.16%, а мужчин - 6.46%
людей без детей - 7.51%, а с 3-мя детьми - 8.18%

Однако это еще не финал. Нужно оценить, можно ли эти проценты переносить на генеральную совокупность (является ли разница статистически значимой)

Возьмем 95% доверительный интервал

Если рассмотреть пол, то доверительный интервал разности получается [−0.03, −0.017], получаем, что разница статистически значима

Если рассмотреть количество детей то доверительный интервал разности получается [−0.037, 0.023], получаем, что нельзя говорить о зависимости между количеством детей и невыплатой кредита.

In [16]:
df = pd.DataFrame()
df['purpose'] = data['purpose']
df['debt'] = data['debt']
print(df.groupby('purpose').sum().sort_values(by='debt'))

                                        debt
purpose                                     
образование                               32
покупка своего жилья                      34
ремонт жилью                              35
на покупку подержанного автомобиля        36
получение образования                     37
дополнительное образование                38
заняться образованием                     39
высшее образование                        40
покупка жилой недвижимости                41
автомобиль                                42
строительство собственной недвижимости    42
недвижимость                              42
приобретение автомобиля                   42
покупка недвижимости                      43
заняться высшим образованием              43
автомобили                                44
на покупку автомобиля                     44
профильное образование                    44
покупка жилья для семьи                   45
на покупку своего автомобиля              46
жилье     

In [17]:
data_debt0=data.loc[data['debt']==0]
data_debt1=data.loc[data['debt']==1]

print('Средний доход тех кто возвращает кредит вовремя - {:.1f}'.format(data_debt0["total_income"].mean()))
print('Средний доход тех у кого есть просрочки по кредиту - {:.1f}'.format(data_debt1["total_income"].mean()))

Средний доход тех кто возвращает кредит вовремя - 167765.5
Средний доход тех у кого есть просрочки по кредиту - 163522.9


In [18]:
print('Средний возраст тех кто возвращает кредит вовремя - {:.1f}'.format(data_debt0["age"].mean()))
print('Средний возраст тех у кого есть просрочки по кредиту - {:.1f}'.format(data_debt1["age"].mean()))

Средний возраст тех кто возвращает кредит вовремя - 43.6
Средний возраст тех у кого есть просрочки по кредиту - 40.3


In [19]:
df = pd.DataFrame()
df['education_id'] = data['education_id']
df['debt'] = data['debt']

sum_df = df.groupby('education_id').sum().sort_values(by='debt')
count_df = df.groupby('education_id').count().sort_values(by='debt')

print ("Процент невыплат кредита в зависимости от образования")
try: 
    sum_df/count_df
except:
    print("Есть нули в count")

print(sum_df/count_df)
    

Процент невыплат кредита в зависимости от образования
                  debt
education_id          
4             0.000000
3             0.109929
2             0.091398
0             0.052852
1             0.089542


In [20]:
df = pd.DataFrame()
df['family_status_id'] = data['family_status_id']
df['debt'] = data['debt']

sum_df = df.groupby('family_status_id').sum().sort_values(by='debt')
count_df = df.groupby('family_status_id').count().sort_values(by='debt')

print ("Процент невыплат кредита в зависимости от семейного положения")
try: 
    sum_df/count_df
except:
    print("Есть нули в count")

print(sum_df/count_df)

Процент невыплат кредита в зависимости от семейного положения
                      debt
family_status_id          
2                 0.065625
3                 0.071130
4                 0.097405
1                 0.092890
0                 0.075202


In [21]:
df = pd.DataFrame()
df['income_type_id'] = data['income_type_id']
df['debt'] = data['debt']

sum_df = df.groupby('income_type_id').sum().sort_values(by='debt')
count_df = df.groupby('income_type_id').count().sort_values(by='debt')

print ("Процент невыплат кредита в зависимости от типа занятости")
try: 
    sum_df/count_df
except:
    print("Есть нули в count")

print(sum_df/count_df)

Процент невыплат кредита в зависимости от типа занятости
                    debt
income_type_id          
0               0.095422
1               0.056017
2               0.073943
3               0.058944
4               0.500000
5               0.000000
6               0.000000
7               1.000000


Таким образом мы получили, что:
* образование играет роль, люди с высшим образованием с большей вероятностью отдадуь кредит, а со средним наоборот
Разчитав доверительный интервал, получаем, что с 95% вероятностью разница между процентом невозврата у людей с высшим и средним образованием находится в интервале (0.029,0.044), а для высшего и неоконченного высшего в интервале - (0.017,0.06).

* семейное положение играет роль. Не женатые люди с большей вероятностью вернут кредит, чем люди, которые женаты или были женаты, 
Например, сравнивая не женатых с вдовцами, получаем, что с 95% уверенностью можно сказать, что вероятность возврата кредита вдовцами больше на процент в интервале (0.013,0.051)

* тип занятости играет роль. Сотрудники самые ненадежные заемщики. Сравнивая их с пенсионерами, компаньенами и госслужащими, получаем, что у них вероятность невозврата больше на % в интервале соответственно: (0.03, 0.048), (0.012, 0.031), (0.023, 0.05)

# Итоги

В результате анализа мы получили следующие результаты.

На вероятность невозврата кредита влияют следующие факторы:
* пол
* образование
* семейное положение
* тип занятости

На вероятность невозврата кредита **НЕ** влияют следующие факторы:
* наличие и количество детей
* возраст
* доход
* цель кредита

Портрет самого надежного заемщика можно описать так - *мужчина, госслужащий с высщим образованием и женатый.*
